# 大規模言語モデル講座 最終課題 LLMコンペティション
下記の３種類のベンチマーク性能を高めよ。ただし、事前学習、FinetuningやRLHF等、プロンプティングの工夫など、授業で学んだことを自由に利用して構わない。モデルや学習に利用できるデータについてはルールを参照のこと。
- Type１：日本語QA
    - 日本に関する知識を答える選択肢問題
    - 出力は選択肢番号(1~5)を出力してください。(int型)
    - 評価は正解率で行います。
- Type２：文章要約
    - 出力は要約文を出力してください。(str型)
    - 評価はROUGE-2 score(F1 score)で行います。
- Type３：Instruction Following
    - 与えられた指示に対して適切な出力を返してください。
    - 出力は生成文を出力してください。(str型)
    - 評価は人手 or 外部の大規模言語モデルを用いて行います。
    - ※本評価に関しては、Type1・Type2ベンチマークでの成績上位者を対象に行います。
        - Type1・Type2成績上位者のうち、Type3のコンペに参加希望の方には人手による評価に協力いただきます

## https://docs.google.com/document/d/1ZyURrEZ-qQulQ3gEQBf6IWHdSTZyU7d-mDAukqzLL-M/edit?usp=sharing  
上記リンク先ドキュメントから特に重要な注意事項を抜粋
- 修了要件の1つとして含まれる
- 開催時期: 2023/09/25 - 2023/10/10 23:59（予定）
    - 締切は2023/10/10 23:59までです。提出の際は以下のファイルをOmnicampusへ提出してもらいます。
        - 提出物１：学習・推論コード(train_and_predict.ipynb)
            - 学習データを作成、読み込んで学習させるコード、評価データを読み込み推論結果のjsonファイルを出力するコード
            - 結果が再現されるか確認に使用します。
            - 成績優秀者の提出コードはコンペ終了後、他の受講生に対して公開させていただきます。
            - starter_code.ipynbを参考にしてください。
        - 提出物２：推論結果(submission.json)
            - id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
- テストデータはtest.jsonというデータをコンペ開始とともに配布します、そのデータにType1, Type2, Type3のデータが全て含まれています
    - 注意: テストデータはコンペ期間中に追加される可能性があります、追加した際はslackにてアナウンスを行うのでご確認ください。
- 配布したGPUリソース(50GPU時間以内)で学習・推論を終えてください
    - なお、GPU時間には最終的な提出物の推論時間を含みます。推論のためのGPU時間を確保しておいてください。※軽いモデルや軽いデータで学習を終えて、1度提出の流れまでは確認することを推奨します。
- ご自身で用意したGPUリソースで学習・試行錯誤をすることは可能ですが、提出する結果の元となるモデルは、配布したGPUリソースで学習を終えてください。
    - 簡単な動作確認などはgoogle colabなどを活用し、大規模な学習をomnicampus上で行うことをオススメします。  
- /workspace/assetsというディレクトリ以下は、インスタンスを閉じた後、再度立ち上げても作業中のファイルやダウンロードや学習させたモデルなどが残るようになっております。
- 受講生間での議論について
    - 生成した出力結果自体を共有するのはやめてください。
    - コンペに関しての受講生同士のディスカッションは#コンペ_受講者ディスカッションで行ってください。
    - コンペのルールに関してご不明な点があれば#コンペ_運営へのルール確認で質問してください。

## Starter Code
testデータを読み込み推論し、提出するファイルを作成する。

In [ ]:
# GPUの確認、枚数に応じて割り当てられた計算資源の消費速度が異なるのでお気をつけください。コンペ期間中は2時間で自動でインスタンスが落ちず、無制限となるため、インスタンスの停止し忘れにもご注意ください。
# 50GPU時間を超えると、インスタンスを立ち上げることができませんのでご注意ください。
!nvidia-smi

import os
# /workspace/assets以下のファイルは永続化されます。作業中のcodeや学習した重みやデータはこちらに保存してください。(容量数TB、IOは遅い)
# /workspace/assets以外の部分のデータ容量は100GBまでです。複数モデルをダウンロードしている場合は容量にご注意ください。
# 具体的には以下のフォルダを適宜消す、あるいは/workspace/assets以下に移動するなどしてください。
os.environ['HF_HOME'] = '/workspace/hf'

## ライブラリのインストール(omnicampus上の環境にないライブラリをインストールした方のみ)

In [ ]:
# 学習・推論コード(train_and_predict.ipynb)の実行に必要なライブラリをインストールする
# pip install new_library==version

## 推論し、提出するファイルを作成する

In [ ]:
import json

# sampleデータ(学習・promptに利用して良いデータ)の確認
sample_data = json.load(open('sample.json', 'r'))
print(len(sample_data))
print(sample_data[:5])
# 推論を行うデータの確認、テストデータはコンペ期間中に追加される可能性があります、追加した際はslackにてアナウンスを行うのでご確認ください。
test_data = json.load(open('test.json', 'r'))
print(len(test_data))
print(test_data[:5])

In [ ]:
# モデルのダウンロードと読み込み
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
model_name = 'matsuo-lab/weblab-10b-instruction-sft'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')

In [ ]:
!nvidia-smi

In [ ]:
import time
import torch
import random

seed = 12
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
random.seed(seed)

start_time = time.time()
# モデル・GPUメモリによって扱える最大トークン数が異なるので注意
# タスクによっては入力文が長いものがあります、どのように対処するかは各自で工夫いただければと思います。
# token長の2乗に計算量とメモリが必要になるので、提出用の推論処理のGPU資源確保の際にはご注意ください。
# max_length = 2048
max_length = 256
test_data = json.load(open('test.json', 'r'))
for data in test_data:
    if data['task_type'] == 'generation':
        text = f"{data['text']} \n\n\n回答:"
        # print(text)
        token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
        max_new_token =int(max_length / 8)
        # print(token_ids.shape)
        if token_ids.shape[1] > max_length:
            token_ids = token_ids[:, -(max_length-max_new_token-1):]

        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )
        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(token_ids[0]):], skip_special_tokens=True)
        data['answer'] = output
    
    elif data['task_type'] == 'summarization':
        text = f"""{data['text']}\n\n\n上記の文章を要約してください。要約:"""
        # print(text)
        token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
        max_new_token =int(max_length / 8)
        # print(token_ids.shape)
        if token_ids.shape[1] > max_length:
            token_ids = token_ids[:, -(max_length-max_new_token-1):]

        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )
        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(token_ids[0]):], skip_special_tokens=True)
        data['answer'] = output
    
    elif data['task_type'] == 'multiple_choice':
        text = f"[問題]:{data['text']} \n\n[選択肢]:[{data['choices'][0]['choice_id']}. {data['choices'][0]['text']}, {data['choices'][1]['choice_id']}. {data['choices'][1]['text']}, {data['choices'][2]['choice_id']}. {data['choices'][2]['text']}, {data['choices'][3]['choice_id']}. {data['choices'][3]['text']}, {data['choices'][4]['choice_id']}. {data['choices'][4]['text']}] \n\n[答えの選択肢番号]:"
        # print(text)
        token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )

        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(token_ids[0]):], skip_special_tokens=True)
        # 出力が1~5の数字になるようにする
        data['answer'] = next((int(char) for char in output if char in '12345'), random.randint(1, 5))
print('推論にかかった秒数', time.time() - start_time)

In [ ]:
# 推論結果の保存、id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
with open('submission.json', 'w') as f:
    json.dump(test_data, f, indent=4)

check_submission_data = json.load(open('submission.json', 'r'))
print(len(check_submission_data), check_submission_data[:5])

## スコア改善のアドバイス
- 他のモデルを使用してみる
    - 上記のサンプルコードで使用しているモデルは[10Bパラメータのモデル](https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft)です、これよりも大きいモデルや別のデータでinstruction-tuning/RLHF学習済みのモデルを使用してみると性能が向上するかもしれません。
        - 無料で使用できるGoogle Colabでもモデルの動作確認を行えるモデルがあるため、重たい計算の必要のない試行錯誤はlocal環境やGoogle Colabで行うことをオススメします。
- promptingを工夫する
    - 現状はシンプルなpromptingになっています。選択したモデルの出力がより良くなるようにfew-shot promptingなどpromptingを工夫してみましょう。
- decodingを工夫しましょう
    - タスクに応じてdecoding手法(greedy-decoding, beam-searchなど)を変更すると性能が向上するかもしれません。
    - 選択肢問題の場合は選択肢を選ぶ際の工夫も有効的かもしれません。
        - https://huggingface.co/blog/evaluating-mmlu-leaderboard
- 学習を工夫しましょう
    - 事前学習・instruction tuning・RLHF学習などを学習データを工夫して行い更なる性能向上を目指しましょう。
        - Day4, Day5, Day6の演習コードや以下のリンクも参考にしてみてください。
            - https://github.com/facebookresearch/llama-recipes
            - https://www.anyscale.com/blog/fine-tuning-llama-2-a-comprehensive-case-study-for-tailoring-models-to-unique-applications
- 受講生同士で知見を共有しましょう
    - submission.jsonの共有は認めておりませんが、データ・実装の共有・prompting・モデル選択の工夫など是非受講生間で議論いただければと思います。
    - コンペに関しての受講生同士のディスカッションは#コンペ_受講者ディスカッションで行ってください。

## 複数GPUによる学習の例
詳細は以下のリンクを参考にしてください
- https://huggingface.co/docs/transformers/accelerate
- https://huggingface.co/docs/transformers/main_classes/deepspeed
- https://pytorch.org/tutorials/distributed/home.html
- https://github.com/huggingface/peft/tree/main/examples/causal_language_modeling 


In [ ]:
# 学習データの作成
import json
sample_data = json.load(open('sample.json', 'r'))
training_data = dict(data=[])

for data in sample_data:
    if data['task_type'] == 'generation':
        text = f"{data['text']} \n\n\n回答:{data['answer']}"
        training_data['data'].append(text)
    elif data['task_type'] == 'summarization':
        text = f"""{data['text']}\n\n\n上記の文章を要約してください。要約:{data['answer']}"""
        training_data['data'].append(text)
    elif data['task_type'] == 'multiple_choice':
        text = f"[問題]:{data['text']} \n\n[選択肢]:[{data['choices'][0]['choice_id']}. {data['choices'][0]['text']}, {data['choices'][1]['choice_id']}. {data['choices'][1]['text']}, {data['choices'][2]['choice_id']}. {data['choices'][2]['text']}, {data['choices'][3]['choice_id']}. {data['choices'][3]['text']}, {data['choices'][4]['choice_id']}. {data['choices'][4]['text']}] \n\n[答えの選択肢番号]:{data['answer']}"
        training_data['data'].append(text)
    else:
        pass
training_data
# 学習データの保存、/workspace/assets以下に保存することで永続化されます。
with open('/workspace/assets/training_data.json', 'w') as f:
    json.dump(training_data, f, indent=4)

In [ ]:
from torch.utils.data import Dataset
class FinetuningDataset(Dataset):
    def __init__(self, data_dict, tokenizer, max_length=256):
        self.model_inputs = []

        for data in data_dict['data']:
            encodings_dict = tokenizer(data)
            # 長い文章を要約するときにどうデータを処理するか、ここでは一旦無視
            if len(encodings_dict['input_ids']) > max_length:
                continue
            else:
                model_inputs = tokenizer(data, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
                del model_inputs['token_type_ids']
                # ここでは全文を予測対象としている(事前学習と同じ設定)
                # instruction tuningの場合はinstruction後の出力を予測対象とするので注意(例: https://github.com/tatsu-lab/stanford_alpaca/blob/main/train.py#L112, SFTTrainer https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)
                labels = tokenizer(data, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
                labels = labels["input_ids"]
                labels[labels == tokenizer.pad_token_id] = -100
                model_inputs["labels"] = labels
                self.model_inputs.append(model_inputs)
    def __len__(self):
        return len(self.model_inputs)

    def __getitem__(self, idx):
        return self.model_inputs[idx]

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

peft_config = LoraConfig(
        task_type="CAUSAL_LM", inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
    )

model_name = 'matsuo-lab/weblab-10b-instruction-sft'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
model = get_peft_model(model, peft_config)

seed = 12
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [ ]:
model.print_trainable_parameters()

In [ ]:
from transformers import AdamW, default_data_collator, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, random_split

batch_size = 1
lr = 5e-4
num_epochs = 3

dataset = FinetuningDataset(training_data, tokenizer)
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
val_dataloader = DataLoader(val_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

optimizer = AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
# 学習
from tqdm import tqdm
import time
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(model.device).squeeze(0) for k, v in batch.items()}
        print("input_ids shape:", batch["input_ids"].shape)
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    for step, batch in enumerate(tqdm(val_dataloader)):
        batch = {k: v.to(model.device).squeeze(0) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()

    eval_epoch_loss = eval_loss / len(val_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

print('学習にかかった秒数', time.time() - start_time)
# 学習済みモデルの保存
model.save_pretrained('/workspace/assets/my_awesome_model_00')

In [ ]:
import time
import torch
import random

seed = 12
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
random.seed(seed)

start_time = time.time()
# モデル・GPUメモリによって扱える最大トークン数が異なるので注意
# タスクによっては入力文が長いものがあります、どのように対処するかは各自で工夫いただければと思います。
# token長の2乗に計算量とメモリが必要になるので、提出用の推論処理のGPU資源確保の際にはご注意ください。
# max_length = 2048
max_length = 256
test_data = json.load(open('test.json', 'r'))
for data in test_data:
    if data['task_type'] == 'generation':
        text = f"{data['text']} \n\n\n回答:"
        # print(text)
        inputs = tokenizer(text, add_special_tokens=False, return_tensors="pt").to(model.device)
        del inputs['token_type_ids']
        max_new_token =int(max_length / 8)
        if inputs.input_ids.shape[1] > max_length:
            inputs.input_ids = inputs.input_ids[:, -(max_length-max_new_token-1):]

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )
        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(inputs.token_ids[0]):], skip_special_tokens=True)
        data['answer'] = output
    
    elif data['task_type'] == 'summarization':
        text = f"""{data['text']}\n\n\n上記の文章を要約してください。要約:"""
        # print(text)
        inputs = tokenizer(text, add_special_tokens=False, return_tensors="pt").to(model.device)
        del inputs['token_type_ids']
        max_new_token =int(max_length / 8)
        if inputs.input_ids.shape[1] > max_length:
            inputs.input_ids = inputs.input_ids[:, -(max_length-max_new_token-1):]

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )
        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(inputs.token_ids[0]):], skip_special_tokens=True)
        data['answer'] = output
    
    elif data['task_type'] == 'multiple_choice':
        text = f"[問題]:{data['text']} \n\n[選択肢]:[{data['choices'][0]['choice_id']}. {data['choices'][0]['text']}, {data['choices'][1]['choice_id']}. {data['choices'][1]['text']}, {data['choices'][2]['choice_id']}. {data['choices'][2]['text']}, {data['choices'][3]['choice_id']}. {data['choices'][3]['text']}, {data['choices'][4]['choice_id']}. {data['choices'][4]['text']}] \n\n[答えの選択肢番号]:"
        # print(text)
        inputs = tokenizer(text, add_special_tokens=False, return_tensors="pt").to(model.device)
        del inputs['token_type_ids']
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                do_sample=False,
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.95
            )

        print(tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True))
        output = tokenizer.decode(output_ids.tolist()[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        # 出力が1~5の数字になるようにする
        data['answer'] = next((int(char) for char in output if char in '12345'), random.randint(1, 5))
print('推論にかかった秒数', time.time() - start_time)

In [ ]:
# 推論結果の保存、id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
with open('trained_model_submission.json', 'w') as f:
    json.dump(test_data, f, indent=4)

check_submission_data = json.load(open('trained_model_submission.json', 'r'))
print(len(check_submission_data), check_submission_data[:5])